<a href="https://colab.research.google.com/github/koreamath/FinalProject-DeepLearning/blob/main/FinalProject_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy


import transformers
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


--2023-12-01 17:42:41--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  11.6MB/s    in 13s     

2023-12-01 17:42:55 (5.97 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2,     # The number of output labels. 2 for binary classification.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
import os
import random

# Function to read files from a directory
def read_files_in_directory(directory):
    files = os.listdir(directory)
    texts = []
    for file_name in files:
        with open(os.path.join(directory, file_name), 'r', encoding='utf8') as file:
            texts.append(file.read())
    return texts

# Load dataset
train_pos = read_files_in_directory('aclImdb/train/pos')
train_neg = read_files_in_directory('aclImdb/train/neg')
test_pos = read_files_in_directory('aclImdb/test/pos')
test_neg = read_files_in_directory('aclImdb/test/neg')

# Combine texts and labels
train_texts = train_pos + train_neg
train_labels = [1] * len(train_pos) + [0] * len(train_neg)  # 1 for positive, 0 for negative

# Optionally shuffle the data
combined = list(zip(train_texts, train_labels))
random.shuffle(combined)
train_texts, train_labels = zip(*combined)

# Now you can use train_test_split as you have done before
from sklearn.model_selection import train_test_split
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_texts, train_labels, random_state=2018, test_size=0.1)


In [ ]:
from transformers import BertTokenizer
import torch

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize and encode sequences in the training set
train_encodings = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=256,  # You can adjust this depending on your needs
    return_tensors='pt'
)

# Tokenize and encode sequences in the validation set
validation_encodings = tokenizer.batch_encode_plus(
    validation_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=256,  # Same as above
    return_tensors='pt'
)

# Extract inputs and masks from the encodings
train_inputs = train_encodings['input_ids']
train_masks = train_encodings['attention_mask']
validation_inputs = validation_encodings['input_ids']
validation_masks = validation_encodings['attention_mask']

# Convert labels to torch tensors
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)



In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # Learning rate
                  eps=1e-8  # Adam's epsilon for numerical stability
                 )
epochs = 4  # You can adjust this according to your needs
from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # Number of warmup steps
                                            num_training_steps=total_steps)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
model = model.to('cuda')
# Training loop
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    model.train()

    total_loss = 0


    for step, batch in enumerate(train_dataloader):
        # Unpack the training batch from our dataloader.
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.to('cuda')
        b_input_mask = b_input_mask.to('cuda')
        b_labels = b_labels.to('cuda')

        # Clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss over all of the batches to calculate the average loss at the end.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)

    # ========================================
    #               Validation
    # ========================================
    print("Running Validation...")

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0  # Initialize the number of evaluation steps


    for batch in validation_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        b_input_ids = b_input_ids.to('cuda')
        b_input_mask = b_input_mask.to('cuda')
        b_labels = b_labels.to('cuda')

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("Validation Accuracy: {0:.2f}".format(avg_val_accuracy))

print("Training complete!")


======== Epoch 1 / 4 ========
Training...
Running Validation...
Validation Accuracy: 0.90
======== Epoch 2 / 4 ========
Training...
Running Validation...
Validation Accuracy: 0.91
======== Epoch 3 / 4 ========
Training...
Running Validation...
Validation Accuracy: 0.91
======== Epoch 4 / 4 ========
Training...
Running Validation...
Validation Accuracy: 0.91
Training complete!


In [ ]:
# Combine test texts and labels
test_texts = test_pos + test_neg
test_labels = [1] * len(test_pos) + [0] * len(test_neg)  # 1 for positive, 0 for negative

# Tokenize and encode sequences in the test set
test_encodings = tokenizer.batch_encode_plus(
    test_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

# Extract inputs and masks from the encodings
test_inputs = test_encodings['input_ids']
test_masks = test_encodings['attention_mask']

# Convert labels to torch tensors
test_labels = torch.tensor(test_labels)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Testing loop (similar to validation loop)
total_test_accuracy = 0

for batch in test_dataloader:
    b_input_ids, b_input_mask, b_labels = batch

    b_input_ids = b_input_ids.to('cuda')
    b_input_mask = b_input_mask.to('cuda')
    b_labels = b_labels.to('cuda')

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    total_test_accuracy += flat_accuracy(logits, label_ids)

avg_test_accuracy = total_test_accuracy / len(test_dataloader)
print("Test Accuracy: {0:.2f}".format(avg_test_accuracy))


Test Accuracy: 0.92


In [ ]:
from sklearn.metrics import precision_recall_fscore_support


# Initialize lists to store predictions and actual labels
all_predictions = []
all_true_labels = []

for batch in test_dataloader:
    b_input_ids, b_input_mask, b_labels = batch

    b_input_ids = b_input_ids.to('cuda')
    b_input_mask = b_input_mask.to('cuda')
    b_labels = b_labels.to('cuda')

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    all_predictions.extend(np.argmax(logits, axis=1).flatten())
    all_true_labels.extend(label_ids.flatten())

# Compute precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(all_true_labels, all_predictions, average='binary')





from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(all_true_labels, all_predictions).ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)


In [ ]:
print("Accuracy: {0:.3f}".format(avg_test_accuracy))
print(f"Precision: {precision:.3f}")
print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"F1 Score: {f1_score:.3f}")

print(f"Recall: {recall:.3f}")

Accuracy: 0.919
Precision: 0.918
Sensitivity: 0.921
Specificity: 0.917
F1 Score: 0.919
Recall: 0.921
